In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random


In [29]:
# Parameters
N = 500  # number of calls
arrival_rate = 1/5  # average 1 call every 5 minutes
service_rate = 1/4  # average 1 service every 4 minutes
MAX_QUEUE_LENGTH = 25

In [31]:
# Agent setup
AGENTS = ["Agent 1", "Agent 2", "Agent 3"]
TOPICS = ["Management", "IT issue"]
RESOLUTION_PROB = {
    ("Agent 1", "Management"): 0.9,
    ("Agent 1", "IT issue"): 0.5,
    ("Agent 2", "Management"): 0.5,
    ("Agent 2", "IT issue"): 0.9,
    ("Agent 3", "Management"): 0.8,
    ("Agent 3", "IT issue"): 0.8,
}


In [33]:
# Satisfaction function
def satisfaction_score(resolved, speed_of_answer, talk_duration):
    if not resolved:
        return np.random.randint(1, 4)
    elif speed_of_answer < 60 and talk_duration < 6:
        return np.random.randint(7, 11)
    elif speed_of_answer < 120:
        return np.random.randint(4, 7)
    else:
        return np.random.randint(1, 4)


In [35]:
# Simulation
data = []
start_time = datetime.combine(datetime.today(), datetime.strptime("09:00", "%H:%M").time())
current_time = start_time
queue = []
active_agents = {agent: start_time for agent in AGENTS}

for call_id in range(1, N + 1):
    inter_arrival = np.random.exponential(1 / arrival_rate)
    current_time += timedelta(minutes=inter_arrival)
    date = current_time.date()
    time = current_time.time()
    topic = random.choice(TOPICS)

    assigned = False
    for agent in AGENTS:
        if active_agents[agent] <= current_time:
            # Assign call
            service_time = np.random.exponential(1 / service_rate)
            resolved_prob = RESOLUTION_PROB[(agent, topic)]
            resolved = "Y" if random.random() < resolved_prob else "N"
            speed_of_answer = random.randint(5, 120)
            talk_duration = round(service_time, 2)
            satisfaction = satisfaction_score(resolved == "Y", speed_of_answer, talk_duration)

            data.append([
                call_id,
                agent,
                date,
                time,
                topic,
                "Y",
                resolved,
                speed_of_answer,
                talk_duration,
                satisfaction
            ])
            active_agents[agent] = current_time + timedelta(minutes=service_time)
            assigned = True
            break

    if not assigned:
        if len(queue) >= MAX_QUEUE_LENGTH:
            data.append([
                call_id,
                "",
                date,
                time,
                topic,
                "N",
                "",
                "",
                "",
                ""
            ])
        else:
            queue.append((call_id, topic, date, time))
            data.append([
                call_id,
                "",
                date,
                time,
                topic,
                "N",
                "",
                "",
                "",
                ""
            ])


In [37]:
# Create DataFrame
columns = ["Call ID", "Agent", "Date", "Time", "Topic", "Answered (Y/N)",
           "Resolved (Y/N)", "Speed of answer", "Talk duration", "Satisfaction rating"]

df_simulated = pd.DataFrame(data, columns=columns)


In [39]:
# Preview dataset
print(df_simulated.head())

# Optional: save to CSV
df_simulated.to_csv("call_centre_simulated_data.csv", index=False)

   Call ID    Agent        Date             Time       Topic Answered (Y/N)  \
0        1  Agent 1  2025-04-19  09:00:35.854243    IT issue              Y   
1        2  Agent 2  2025-04-19  09:04:21.340748    IT issue              Y   
2        3  Agent 3  2025-04-19  09:04:59.905884    IT issue              Y   
3        4  Agent 3  2025-04-19  09:06:12.614602  Management              Y   
4        5  Agent 1  2025-04-19  09:10:24.060129  Management              Y   

  Resolved (Y/N) Speed of answer Talk duration Satisfaction rating  
0              N             106          8.43                   2  
1              Y               8          2.64                  10  
2              N              22          0.38                   1  
3              Y              49         10.97                   6  
4              Y              77          6.68                   5  
